In [4]:
import os
import tensorflow as tf
from tensorflow.python.lib.io import file_io

print('\n{:<15}{}'.format('tensorlow', tf.__version__))



print('init variables')

# global variables
TFR_TRAIN = 'train.tfrecord'
TFR_VALID = 'valid.tfrecord'
TFR_TEST = 'test.tfrecord'
BUCKET = 'gs://robolab/'

# image and classes
NUM_CLASSES = 2
IMG_HEIGHT = 80
IMG_WIDTH = 71

# model dir
OUTDIR = BUCKET + 'output'

# hypers
BATCH_SIZE = 10
EPOCHS = 1
LR = 0.0001



print('define TFR parsers')

def parser(serialized_example):

    features = tf.parse_single_example(
        serialized_example,
        features={
            'image_raw': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.string)})

    image = tf.decode_raw(features['image_raw'], tf.float32)
    image.set_shape([IMG_HEIGHT * IMG_WIDTH])

    label = tf.decode_raw(features['label'], tf.int32)
    label.set_shape([1])

    return image, label

def test_parser(serialized_example):

    features = tf.parse_single_example(
        serialized_example,
        features={'image_raw': tf.FixedLenFeature([], tf.string),
                 'image_id': tf.FixedLenFeature([], tf.string)})

    image = tf.decode_raw(features['image_raw'], tf.float32)
    image.set_shape([IMG_HEIGHT * IMG_WIDTH])

    return image


print('define input functions')

def train_input_fn():

    # get dataset from tf_record
    dataset = tf.data.TFRecordDataset(BUCKET + TFR_TRAIN)

    # map parser over dataset samples
    dataset = dataset.map(parser)
    dataset = dataset.shuffle(1000)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.repeat(1)
    iterator = dataset.make_one_shot_iterator()

    features, labels = iterator.get_next()

    return features, labels

def eval_input_fn():

    # get dataset from tf_record
    dataset = tf.data.TFRecordDataset(BUCKET + TFR_TRAIN)

    # map parser over dataset samples
    dataset = dataset.map(parser)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.repeat(1)
    iterator = dataset.make_one_shot_iterator()

    features, labels = iterator.get_next()

    return features, labels

def valid_input_fn():

    # get dataset from tf_record
    dataset = tf.data.TFRecordDataset(BUCKET + TFR_VALID)

    # map parser over dataset samples
    dataset = dataset.map(parser)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.repeat(1)
    iterator = dataset.make_one_shot_iterator()

    features, labels = iterator.get_next()

    return features, labels

def predict_input_fn():

    # get dataset from tf_record
    dataset = tf.data.TFRecordDataset(BUCKET + TFR_TEST)

    # map parser over dataset samples
    dataset = dataset.map(test_parser)
    dataset = dataset.batch(1)
    dataset = dataset.repeat(1)
    iterator = dataset.make_one_shot_iterator()

    features = iterator.get_next()

    return features

def get_image_id(serialized_example):

    features = tf.parse_single_example(
        serialized_example,
        features={'image_raw': tf.FixedLenFeature([], tf.string),
                  'image_id': tf.FixedLenFeature([], tf.string)})
    
    return features['image_id']


print('define model')

def cnn_model_fn(features, labels, mode):
    
    input_layer = tf.reshape(features, [-1, IMG_HEIGHT, IMG_WIDTH])
    input_layer = tf.expand_dims(input_layer, axis=3)

    conv_layer_1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=8,
        kernel_size=[2, 2],
        padding='same',
        activation=tf.nn.relu)

    pool_layer_1 = tf.layers.max_pooling2d(
        inputs=conv_layer_1,
        pool_size=[2, 2],
        strides=2,
        padding='same')

    conv_layer_2 = tf.layers.conv2d(
        inputs=pool_layer_1,
        filters=32,
        kernel_size=[2, 2],
        padding='same',
        activation=tf.nn.relu)

    pool_layer_2 = tf.layers.max_pooling2d(
        inputs=conv_layer_2,
        pool_size=[2, 2],
        strides=2,
        padding='same')

    reshape_layer = tf.layers.flatten(pool_layer_2)

    dense_layer = tf.layers.dense(
        inputs=reshape_layer,
        units=256,
        activation=tf.nn.relu)
    
    is_train = False

    if mode == tf.estimator.ModeKeys.TRAIN:
        is_train = True

    dropout_layer = tf.layers.dropout(
        inputs=dense_layer,
        rate=0.2,
        training=is_train)

    logits_layer = tf.layers.dense(
        inputs=dropout_layer,
        units=NUM_CLASSES)

    predictions = {
        'classes':tf.argmax(logits_layer, axis=1),
        'probabilities':tf.nn.softmax(logits_layer, axis=1)}

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    loss = tf.losses.sparse_softmax_cross_entropy(
        labels=labels,
        logits=logits_layer)

    accuracy = tf.metrics.accuracy(
        labels=labels,
        predictions=tf.argmax(logits_layer, axis=1),
        name='accu_op')

    if mode == tf.estimator.ModeKeys.TRAIN:
        train_optimizer = tf.train.AdamOptimizer(learning_rate=LR).minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_optimizer)

    # mode = EVAL
    eval_metric_ops = {'accuracy':accuracy}

    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)



print('define train function')

def train_and_evaluate(estimator, epochs=1):

    all_train_log = []
    all_valid_log = []

    for epoch in range(epochs):

        estimator.train(input_fn=train_input_fn)

        train_log = estimator.evaluate(input_fn=eval_input_fn)
        print('epoch: {} of {}'.format(epoch + 1, epochs))
        print('train: acc={:.3f} loss={:.3f}'.format(train_log['accuracy'], train_log['loss']))
        
        valid_log = estimator.evaluate(input_fn=valid_input_fn)
        print('valid: acc={:.3f} loss={:.3f}'.format(valid_log['accuracy'], valid_log['loss']))

        all_train_log.append(train_log)
        all_valid_log.append(valid_log)

    return all_train_log, all_valid_log



print('start train...')

cnn_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir=OUTDIR)

train_log, valid_log = train_and_evaluate(cnn_classifier, epochs=EPOCHS)

print('train finished')



print('start predictions')

predict_generator = cnn_classifier.predict(input_fn=predict_input_fn)

preds = []
predict_dictlist = []

while True:
    item = next(predict_generator, None)
    if item == None:
        break
    predict_dictlist.append(item)

for i in range(len(predict_dictlist)):
    class_ = predict_dictlist[i]['classes']
    preds.append(class_)

image_ids = []
image_id_tensors = []
record_iterator = tf.python_io.tf_record_iterator(path=BUCKET + TFR_TEST)

for string_record in record_iterator:
    image_id_tensors.append(get_image_id(string_record))
    
with tf.Session() as sess:
     image_ids = sess.run(image_id_tensors)

print('prediction finished')


print('create csv file...')

sub_filename = 'submission.csv'
submission_filepath = os.path.join(BUCKET, 'subs', sub_filename)

preds = [i + 1 for i in preds]

with file_io.FileIO(submission_filepath, mode='w') as fout:
    fout.write('filename,label\n')
    for pred, image_id in zip(preds, image_ids):
        image_id = image_id.decode('utf-8')
        fout.write('{},{}\n'.format(image_id, pred))
        
print('done')


tensorlow      1.12.0
numpy          1.15.4

init variables
define TFR parsers
define input functions
define model
define train function
start train...
epoch: 1 of 1
train: acc=0.905 loss=0.228
valid: acc=0.900 loss=0.242
train finished
start predictions
prediction finished
create csv file...
done
